In [18]:
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND*5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)
session.headers['User-agent'] = 'n.finance/1.0'

In [19]:
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf

stocks = ['2330.TW', '2317.TW', '1216.TW']
tickers = yf.Tickers(stocks, session=session)

In [20]:
close = tickers.history(start='2013-12-31', end='2016-12-31', auto_adjust=False)["Close"]
close

[*********************100%%**********************]  3 of 3 completed


,1216.TW,2317.TW,2330.TW
Date,,,
2013-12-31,48.711903,77.400276,105.5
2014-01-02,48.984035,77.690163,104.5
2014-01-03,48.530479,76.723869,102.5
2014-01-06,48.530479,76.337357,102.5
2014-01-07,48.439766,76.337357,102.0
...,...,...,...
2016-12-26,52.700001,104.125000,179.5
2016-12-27,53.000000,104.375000,180.0
2016-12-28,52.900002,105.000000,182.0


In [21]:
returns = close.pct_change()
returns.dropna()

,1216.TW,2317.TW,2330.TW
Date,,,
2014-01-02,0.005587,0.003745,-0.009479
2014-01-03,-0.009259,-0.012438,-0.019139
2014-01-06,0.000000,-0.005038,0.000000
2014-01-07,-0.001869,0.000000,-0.004878
2014-01-08,-0.003745,0.006329,0.019608
...,...,...,...
2016-12-26,0.009579,0.007255,0.005602
2016-12-27,0.005693,0.002401,0.002786
2016-12-28,-0.001887,0.005988,0.011111
